# Transportation Problem

\begin{align*}
\text{Minimize} \quad & \sum_{i=1}^{m} \sum_{j=1}^{n} c_{ij}x_{ij} & \\
\text{Subject to} \quad & \sum_{j=1}^{n} x_{ij} \leq S_i & (i = 1, 2, \dots, m) \\
& \sum_{i=1}^{m} x_{ij} \geq D_j & (j = 1, 2, \dots, n) \\
& x_{ij} \geq 0 & (i = 1, 2, \dots, m, j = 1, 2, \dots, n)
\end{align*}


### Parameters
- $m$ and $n$ are the number of supply and demand locations respectively.
- $c_{ij}$ is the cost of shipping one unit of goods from supply location $i$ to demand location $j$.
- $S_i$ is the number of units of goods that can be shipped from supply location $i$.
- $D_j$ is the number of units of goods that need to be shippedto demand location $j$.

### Decision Variables
- $x_{ij}$: the number of units of goods shipped from supply location $i$ to demand location $j$.  




In [213]:
#Import the neccessary libraries and modules
import gurobipy as gp
from gurobipy import  Model, GRB, quicksum
from array import array
from gurobipy import *
import numpy as np
import time
import pandas as pd

In [214]:
#Loading the data from excel file 'Transportatioin.xlsx'
df = pd.read_excel('Transportation.xlsx' , sheet_name = 0)

In [215]:
df

,City,Moncton,Sydney,Ottawa,Truro,Saint John,PEI,Hamilton,Waterloo,Capacity
0,Halifax,98,65,90,68,52,134,108,118,187.0
1,London,51,92,66,123,93,138,130,57,893.0
2,Ajax,86,112,100,85,147,142,83,84,488.0
3,Toronto,98,72,148,149,101,75,55,144,769.0
4,Montreal,110,50,83,105,142,66,68,65,663.0
5,Demand,345,398,462,287,532,196,765,15,NaN


In [216]:
#Extracting the necessary data from data frame
array_value = df.iloc[0:5, 1:9].values

In [217]:
#Creating source list-cost
origin = df.iloc[0:5, 9].tolist()
origin

[187.0, 893.0, 488.0, 769.0, 663.0]

In [218]:
#Creating destination list-cost
destination = df.iloc[5, 1:9].tolist()
destination

[345, 398, 462, 287, 532, 196, 765, 15]

In [219]:
O = df.iloc[0:5, 0].tolist()
O

['Halifax', 'London', 'Ajax', 'Toronto', 'Montreal']

In [220]:
D = list(df.columns.values)[1:9]
D

['Moncton',
 'Sydney',
 'Ottawa',
 'Truro',
 'Saint John',
 'PEI',
 'Hamilton',
 'Waterloo']

In [221]:
#Creating dictonaries and list to store the data
list1 = [(i, j) for i in O for j in D]

In [222]:
list1

[('Halifax', 'Moncton'),
 ('Halifax', 'Sydney'),
 ('Halifax', 'Ottawa'),
 ('Halifax', 'Truro'),
 ('Halifax', 'Saint John'),
 ('Halifax', 'PEI'),
 ('Halifax', 'Hamilton'),
 ('Halifax', 'Waterloo'),
 ('London', 'Moncton'),
 ('London', 'Sydney'),
 ('London', 'Ottawa'),
 ('London', 'Truro'),
 ('London', 'Saint John'),
 ('London', 'PEI'),
 ('London', 'Hamilton'),
 ('London', 'Waterloo'),
 ('Ajax', 'Moncton'),
 ('Ajax', 'Sydney'),
 ('Ajax', 'Ottawa'),
 ('Ajax', 'Truro'),
 ('Ajax', 'Saint John'),
 ('Ajax', 'PEI'),
 ('Ajax', 'Hamilton'),
 ('Ajax', 'Waterloo'),
 ('Toronto', 'Moncton'),
 ('Toronto', 'Sydney'),
 ('Toronto', 'Ottawa'),
 ('Toronto', 'Truro'),
 ('Toronto', 'Saint John'),
 ('Toronto', 'PEI'),
 ('Toronto', 'Hamilton'),
 ('Toronto', 'Waterloo'),
 ('Montreal', 'Moncton'),
 ('Montreal', 'Sydney'),
 ('Montreal', 'Ottawa'),
 ('Montreal', 'Truro'),
 ('Montreal', 'Saint John'),
 ('Montreal', 'PEI'),
 ('Montreal', 'Hamilton'),
 ('Montreal', 'Waterloo')]

In [223]:
#Arc cost - flatten()
cost = np.array(array_value).flatten()

In [224]:
#printing the cost
cost

array([ 98,  65,  90,  68,  52, 134, 108, 118,  51,  92,  66, 123,  93,
       138, 130,  57,  86, 112, 100,  85, 147, 142,  83,  84,  98,  72,
       148, 149, 101,  75,  55, 144, 110,  50,  83, 105, 142,  66,  68,
        65])

In [225]:
cost_values = {list1[i]: cost[i] for i in range(len(list1))}
cost_values

{('Halifax', 'Moncton'): 98,
 ('Halifax', 'Sydney'): 65,
 ('Halifax', 'Ottawa'): 90,
 ('Halifax', 'Truro'): 68,
 ('Halifax', 'Saint John'): 52,
 ('Halifax', 'PEI'): 134,
 ('Halifax', 'Hamilton'): 108,
 ('Halifax', 'Waterloo'): 118,
 ('London', 'Moncton'): 51,
 ('London', 'Sydney'): 92,
 ('London', 'Ottawa'): 66,
 ('London', 'Truro'): 123,
 ('London', 'Saint John'): 93,
 ('London', 'PEI'): 138,
 ('London', 'Hamilton'): 130,
 ('London', 'Waterloo'): 57,
 ('Ajax', 'Moncton'): 86,
 ('Ajax', 'Sydney'): 112,
 ('Ajax', 'Ottawa'): 100,
 ('Ajax', 'Truro'): 85,
 ('Ajax', 'Saint John'): 147,
 ('Ajax', 'PEI'): 142,
 ('Ajax', 'Hamilton'): 83,
 ('Ajax', 'Waterloo'): 84,
 ('Toronto', 'Moncton'): 98,
 ('Toronto', 'Sydney'): 72,
 ('Toronto', 'Ottawa'): 148,
 ('Toronto', 'Truro'): 149,
 ('Toronto', 'Saint John'): 101,
 ('Toronto', 'PEI'): 75,
 ('Toronto', 'Hamilton'): 55,
 ('Toronto', 'Waterloo'): 144,
 ('Montreal', 'Moncton'): 110,
 ('Montreal', 'Sydney'): 50,
 ('Montreal', 'Ottawa'): 83,
 ('Montreal',

In [226]:
O_origin = {O[i] : origin[i] for i in range(len(origin))}
O_origin

{'Halifax': 187.0,
 'London': 893.0,
 'Ajax': 488.0,
 'Toronto': 769.0,
 'Montreal': 663.0}

In [227]:
D_destination = {D[i] : destination[i] for i in range(len(destination))}
D_destination

{'Moncton': 345,
 'Sydney': 398,
 'Ottawa': 462,
 'Truro': 287,
 'Saint John': 532,
 'PEI': 196,
 'Hamilton': 765,
 'Waterloo': 15}

In [228]:
#Making sure that Total Supply = Demand

In [229]:
d=len(O_origin.values())
if sum(O_origin.values()) < sum(D_destination.values()):
    diff = sum(D_destination.values()) - sum(O_origin.values()) 
    O_origin[d] += diff

In [230]:
#Initialize the model
model = gp.Model('transport model')

In [231]:
#Adding the decision variables 
x = {}
for i in O:
    for j in D:
        x[i,j] = model.addVar(vtype = 'I', name = 'x(%s,%s)' %(i,j))

In [232]:
#Adding constraints to the model
for i in O:
    model.addConstr(quicksum(x[i,j] for j in D) <= O_origin[i], name = 'Origin(%s)' % i)

In [233]:
for j in D:
    model.addConstr(quicksum(x[i,j] for i in O) >= D_destination[j], name = 'Destination(%s)' % j)

In [234]:
#Set objective function
model.setObjective(quicksum(x[i,j] * transit_cost[i,j] for i,j in transit), GRB.MINIMIZE)

In [235]:
#Update the model
model.update()

In [236]:
model.write('supply_demand2.lp')

In [237]:
#Optimize the model
start = time.time()
model.optimize()
end = time.time()

def print_solution(model):
    if model.status != GRB.OPTIMAL:
        print('Model is not optimized!')
    else:
        print('Optimal Value: ' , model.ObjVal)
        print('Solution time: ' , end - start, 'seconds')
        print('==========================================')
        vars = model.getVars()
        values = model.getAttr('x', vars)
        for var, val in zip(vars, values):
            if val > 1e-6:
                print(f"{var.varName}: {val}")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13 rows, 40 columns and 80 nonzeros
Model fingerprint: 0xf6fa9e87
Variable types: 0 continuous, 40 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 9e+02]
Found heuristic solution: objective 256232.00000
Presolve time: 0.00s
Presolved: 13 rows, 40 columns, 80 nonzeros
Variable types: 0 continuous, 40 integer (0 binary)

Root relaxation: objective 1.979610e+05, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    197961.00000 197961.000  0.00%     -    0s

Explored 1 nodes (12 simplex iterations) in 0.01 seconds (

In [238]:
print_solution(model)

Optimal Value:  197961.0
Solution time:  0.019556760787963867 seconds
x(Halifax,Saint John): 187.0
x(London,Moncton): 345.0
x(London,Ottawa): 207.0
x(London,Saint John): 341.0
x(Ajax,Ottawa): 201.0
x(Ajax,Truro): 287.0
x(Toronto,Saint John): 4.0
x(Toronto,Hamilton): 765.0
x(Montreal,Sydney): 398.0
x(Montreal,Ottawa): 54.0
x(Montreal,PEI): 196.0
x(Montreal,Waterloo): 15.0
